# Joining Data
In this exercise, you join data from multiple data frames together. It is common to import data from multiple SQL tables which are related to each other. Given multiple data frames with one or more related column, you use various join functions from the dplyr tidyverse package to combine them together. 


## R Features
* library()
* dbConnect()
* print()
* dbGetQuery()
* glimpse()
* nrow() 
* distinct()
* inner_join()
* left_join()
* right_join()
* full_join()
* rename()
* select()
* semi_join()
* anti_join()
* filter()
* dbDisconnect()

## Datasets
* AdventureWorks

In [ ]:
# Load libraries
library(___) # odbc
library(___) # DBI
library(___) # tidyverse

## Import several related SQL tables
Connect to the AdventureWorks SQL database and download some tables

In [ ]:
# Connection string info
# Already completed, just run the code block
# Everyone uses the same SQL credentials
driver_name <- "ODBC Driver 13 for SQL Server"
server_name <- "uwc-sqlserver.clients.uw.edu"
database_name <- "AdventureWorks2016CTP3" 
user_id <- "sqlstudentreader"
password <- "PA6aX2gAhe4hE!ru$6atru"

# Connect to the database
# Store connection in conn variable
conn <- ___(odbc::odbc(), 
                  driver = driver_name, 
                  server = server_name, 
                  database = database_name,
                  uid = user_id,
                  pwd = password)

# Print the connection object
print(___)

## Sales Order Tables
Table: Sales.SalesOrderHeader
* Primary Key: SalesOrderID (PK)
* Foreign Keys: CustomerID, SalesPersonID, TerritoryID, BillToAddressID, ShipToAddressID, ShipMethodID, CurrencyRateID

Table: Sales.SalesOrderDetail
* Primary Key: SalesOrderDetail
* Foreign Keys: SalesOrderID, ProductID, SpecialOfferID

In [ ]:
# Get Sales.SalesOrderHeader
sql_select <- "SELECT * FROM ___"
df_sales_order_header <- conn %>% 
   ___(___)

# Get Sales.SalesOrderDetail
sql_select <- "SELECT * FROM ___"
df_sales_order_detail <- conn %>% 
   ___(___)

# Glimpse results
glimpse(___)
glimpse(___)

Notice the number of rows for the two tables. Which table has the larger number of rows? Why? 

Notice that the SalesOrderID column is present in both tables. In SalesOrderHeader it is the primary key meaning that there is only one row for each unique value. In SalesOrderDetail it is a foreign key and multiple rows may have the same value.

## Distinct()

Select distinct/unique rows. Retain only unique/distinct rows from an input tbl. This is similar to unique.data.frame, but considerably faster. 

distinct(.data, ..., .keep_all = FALSE)

In [ ]:
# View help on distinct()
?___

In [ ]:
# Get the row count for 
# Sales.SalesOrderHeader
# Hint: nrow()
df_sales_order_header %>% 
   ___()

# Get the distinct row count for 
# Sales.SalesOrderHeader/SalesOrderID
# Hint: distinct() and nrow()
df_sales_order_header %>% 
   ___(___) %>% 
   ___()

Notice that the overall row count and the distinct row count for SalesOrderID is the same. This is required for the primary key.

In [ ]:
# Get the row count for 
# Sales.SalesOrderDetail
# Hint: nrow()
df_sales_order_detail %>% 
   ___()

# Get the distinct row count for 
# Sales.SalesOrderDetail/SalesOrderID
# Hint: distinct() and nrow()
df_sales_order_detail %>% 
   ___(___) %>% 
   ___()

Notice that the overall row count is much larger than the distinct values of SalesOrderID. This is because SalesOrderID is a foreign key with values used more than once. The grain of the sales order *detail* table is one row per sales transaction AND product ID. This allows storing of quantity of each product purchased. In contrast, the sales order *header* does not contain product ID or its quantity purchased. 

Also notice that the unique values matched the number of rows for the sales order header table. This is a sign of clean data with SQL referrential integrity enabled to ensure that they match. There can be no header row without a detail row. There can be no detail row without a header row.

## Joins
Join two tables together. 

### Usage
inner_join(x, y, by = NULL, copy = FALSE, suffix = c(".x", ".y"), ...)

left_join(x, y, by = NULL, copy = FALSE, suffix = c(".x", ".y"), ...)

right_join(x, y, by = NULL, copy = FALSE, suffix = c(".x", ".y"), ...)

full_join(x, y, by = NULL, copy = FALSE, suffix = c(".x", ".y"), ...)

semi_join(x, y, by = NULL, copy = FALSE, ...)

anti_join(x, y, by = NULL, copy = FALSE, ...)

### Arguments
* x, y tbls to join
* by a character vector of variables to join by. If NULL, the default, join will do a natural join, using all variables with common names across the two tables. A message lists the variables so that you can check they're right (to suppress the message, simply explicitly list the variables that you want to join). To join by different variables on x and y use a named vector. For example, by = c("a" = "b") will match x.a to y.b.

### Join types
Currently dplyr supports four join types: 
* inner_join return all rows from x where there are matching values in y, and all columns from x and y. If there are multiple matches between x and y, all combination of the matches are returned.
* left_join return all rows from x, and all columns from x and y. Rows in x with no match in y will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.
* right_join return all rows from y, and all columns from x and y. Rows in y with no match in x will have NA values in the new columns. If there are multiple matches between x and y, all combinations of the matches are returned.
* semi_join return all rows from x where there are matching values in y, keeping just columns from x. A semi join differs from an inner join because an inner join will return one row of x for each matching row of y, where a semi join will never duplicate rows of x.
* anti_join return all rows from x where there are not matching values in y, keeping just columns from x.
* full_join return all rows and all columns from both x and y. Where there are not matching values, returns NA for the one missing.

In [ ]:
# View help on inner join
?___

In [ ]:
# Join Sales.SalesOrderHeader and Sales.SalesOrderDetails
# using inner_join() 
# using automatic join column (omit the 'by' parameter)
# store as df_sales_order_inner
df_sales_order_inner <- df_sales_order_header %>%
   ___(___)

# Glimpse result
glimpse(___)

Notice the error message in red and the number of rows returned. inner_join() selected all the columns that had matching names between the two dataframes as its best guess for which columms to join on. Although we wanted to join on the primary key which was just SalesOrderID, there were two more columns in common between the two tables, rowguid and ModifiedDate. The values of all three columns needed to match exactly in order to return any rows. Thus no rows were returned because there were no exact matches. 

Notice the columns are all the columns from the left table plus all the columns from the right table...except there is only one set of join columns. There is only one instance of SalesOrderID and the other two join columns. This makes sense because the join is looking for matching values from the two tables. If there was a SalesOrderID from the left table and a different SalesOrderID from the right table, they would always contain the same value so there is no need to include the join columns more than once.

## Inner join vs left join vs right join vs full join
In the above example, no rows were returned because of two reasons. The first is that it used three columns to join with, called a composite key which is a primary key that contains multiple columns. The second is that inner join was used which requires exact matches from the left and the right tables. 

We will fix the join criteria issue in a moment, but first let's illustrate the difference among the different join types for the same join columns for these two tables.

In [ ]:
# Join Sales.SalesOrderHeader and Sales.SalesOrderDetails
# using left_join() 
# using automatic join column (omit the 'by' parameter)
# store as df_sales_order_left
df_sales_order_left <- df_sales_order_header %>%
   ___(___)

# Glimpse result
glimpse(___)

Notice the number of rows returned from the left join. Does this match one of the source tables, SalesOrderHeader or SalesOrderDetail?

It should match SalesOrderHeader because it was the *left* table in the join AND there were no matches to the right table. Left joins preserve all the rows in the left table. 

Also notice the values of the columns from SalesOrderDetail. They are all NA. This is also common because if there isn't a match between the left and right tables, the row from the left table is inserted and NA values are used as placeholders for the right table.

In [ ]:
# Join Sales.SalesOrderHeader and Sales.SalesOrderDetails
# using right_join() 
# using automatic join column (omit the 'by' parameter)
# store as df_sales_order_right
df_sales_order_right <- df_sales_order_header %>%
   ___(___)

# Glimpse result
glimpse(___)

How many rows were returned? Which table does it match, SalesOrderHeader or SalesOrderDetail? 

Similar to left join, right join retained all the rows from the right table and filled in NA values for the left table when there are no matches. 

In [ ]:
# Join Sales.SalesOrderHeader and Sales.SalesOrderDetails
# using full_join() 
# using automatic join column (omit the 'by' parameter)
# store as df_sales_order_full
df_sales_order_full <- df_sales_order_header %>%
   ___(___)

# Glimpse result
glimpse(___)

How many rows were returned? Which table does it match, SalesOrderHeader or SalesOrderDetail? 

The row count is larger than either table. Why? Because full join has charactistics of both left and right joins in that all the rows from the left table and all the rows from the right table are present in the result. Since there were no matches at all between the two table it returned all the rows from SalesOrderHeader plus all the rows from SalesOrderDetail. It filled in NA values as appropriate.

## Using by
The reason there were no matches were because by default, the join functions look for the same column names between the two tables and use that. We can specify the by parameter to override this default behavior. 

There are three different syntax for by:
1. by = "column name" - use this for the same single column name in both tables
2. by = c("col_1", "col_2") - use this for the same multiple column names (composite primary keys)
3. by = c("col_1_left" = "col_1_right", "col_2_left" = "col_2_right") - use this when the names don't match between the two tables. It also supports composite primary keys.

In [ ]:
# Join Sales.SalesOrderHeader and Sales.SalesOrderDetails
# using inner_join() 
# using by = "SalesOrderID"
# store as df_sales_order
df_sales_order <- df_sales_order_header %>%
   ___(___, by = ___)

# Glimpse result
glimpse(___)

How many rows of data were returned? Does it match SalesOrderHeader or SalesOrderDetail? 

Since there were matches for each and every row of SalesOrderDetail, then the row count matches the number of rows for SalesOrderDetail. 

Look at the columns closely and compare the number of columns to the previous joins. Do you see any additional columns?

Recall that rowguid and ModifiedDate had the same name in both tables and that the auto join feature used these columns in its join earlier. Since we manually specified the join column using the 'by' parameter and didn't include these two columns, it created a naming conflict when combining the two tables together. In order to include the conflicting columns from the left table and the right table, it renamed the columns appending a suffix. A suffix of '.x' means it came from the left table. A suffix of '.y' means it came from the right table.

In [ ]:
# Rename the conflicting .x and .y columns
# using the naming convention <table>_<column>
# update df_sales_order
df_sales_order <- df_sales_order %>%
   rename(___)

# Glimpse result
glimpse(___)

Notice the renamed columns and the '.x' and '.y' are gone. Renaming columns with the table name helps data traceablity and troubleshooting. Alternatively, if these fields were not needed for the analysis, then consider not importing these columns from SQL in the first place. This saves computer resources and avoids unnecesary clean up work later. 

## Single vs Multiple Dataframes
You can continue to join tables and create a single table at the lowest data grain with a large number of columns. This is useful for machine learning algorithms where all the data it processes must be in a single table. 

For human analysis, however, it might be more effective to have several data frames you can easily join together as required. Consider a data warehouse star schema as a guideline for how to design your dataframes.

In [ ]:
# Get Production.Product
# exclude metadata columns: rowguid, ModifiedDate
sql_select <- "SELECT * FROM ___"
df_product <- conn %>% 
   ___(___) %>% 
   select(___, ___)

# Get Sales.SalesTerritory
# exclude metadata columns: rowguid, ModifiedDate
sql_select <- "SELECT * FROM ___"
df_sales_territory <- conn %>% 
   ___(___) %>% 
   select(___, ___)

# Glimpse results
glimpse(___)
glimpse(___)

In [ ]:
# left join df_sales_order with
# df_product and df_sales_territory
# store df_sales_order_complete
# do not specify the by parameter
df_sales_order_complete <- df_sales_order %>% 
   ___(___) %>% 
   ___(___)

# Glimpse result
glimpse(___)

Notice the warning message from the automatic by parameter. Were these the join columns you expected? 

This is a convenient way to help determine the by parameter and alert you to any potential issues. 

In [ ]:
# left join df_sales_order with
# df_product and df_sales_territory
# rename the Name columns to <table>_Name before joining
# store df_sales_order_complete
# do not specify the by parameter
df_sales_order_complete <- df_sales_order %>% 
   ___(___ %>% rename(___)) %>% 
   ___(___ %>% rename(___))

# Glimpse result
glimpse(___)

Are the rows of data what you expected? 

Adding in lookup or dimension tables shouldn't increase the row count from the underlying transaction or fact table. Always look at the row count from joins. The row count may alert you to when data is messy or the grain of the table is not what you expected. Using a left join instead of an inner join ensure that no transaction rows get removed. 

Notice that you can alter the left and right join tables inline in the code. Once the Name columns were renamed before joining, the auto join feature worked as expected. If the warning message bothers you, you can always specify the join criteria explicitly. If you practice defensive programming, you would be explicit whenever possible as opposed to relying on automatic behavior which could have unexpected results if the input data changes.

## Semi_join() and Anti_join()
If you don't want to join all the data into a single table, you will have some sort of related dataframes that you may need to join together for analysis purposes. semi_join() and anti_join() are useful in situations where you want to filter one dataframe by some criteria in another dataframe but do *not* need the data from the other dataframe. 

semi and anti joins perform the join but don't add any of the rows from the right table. They simply filter the left table. A semi join retains rows when there is a match with the right table. anti join has the opposite behavior, retains only the rows *not* in the right table.

In [ ]:
# semi join df_sales_order
# with df_sales_territory
# glimpse result, do not store in variable
df_sales_order %>% ___(___) %>% 
   glimpse()

Is the row count what you expected? Are there any SalesTerritory columns part of the result? 

Semi join matched on TerritoryID for each and every row. This is an indication of clean source data. 

In [ ]:
# anti join df_sales_order
# with df_sales_territory
# glimpse result, do not store in variable
df_sales_order %>% ___(___) %>% 
   glimpse()

Are the rows returned what you expected? 

Since anti_join is the opposite from semi_join, if semi_join matched all the rows then anti_join matched 0 rows. 

In [ ]:
# semi join df_sales_order
# with df_sales_territory
# inline filtered on CountryRegionCode == "US"
# glimpse result, do not store in variable
df_sales_order %>% ___(___ %>% filter(___)) %>% 
   glimpse()

Is the row count what you expected? 

There are fewer rows in the result reflecting the filtering of only US sales. 

In [ ]:
# anti join df_sales_order
# with df_sales_territory
# inline filtered on Group == "North America"
# glimpse result, do not store in variable
df_sales_order %>% ___(___ %>% filter(___)) %>% 
   glimpse()

Is the row count what you expected? 

These results are the sales from all territory groups except for North America.

In [ ]:
# Close SQL connection
conn %>% ___()

It is good practice to close the connection when done to conserve resources. Often the connection will automatically close after a timeout duration. If it closes before you wanted it to, just run your dbConnect code again.

## Summary
Joining is a common data wrangling task as you accumulate more and more dataframes of data. You use joining to organize the data into a design that makes your analysis more agile. You also use joining as a way to explore and filter the data as part of data exploration.